In [1]:
from PIL import Image,ImageDraw
import json
import numpy as np
from matplotlib import pyplot as plt
from math import floor
Image.MAX_IMAGE_PIXELS = None
import urllib.request, json
import requests
from io import BytesIO
import os
from urllib.parse import urlparse

import logging
import boto3
from botocore.exceptions import ClientError

# check Pillow version number
import PIL
print('Pillow Version:', PIL.__version__)
import numpy as np
import os
from pathlib import Path
import re
import json
# load and show an image with Pillow
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt

Pillow Version: 8.1.0


In [2]:
def upload_file(file_path:os.path, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """
    #print("S3 parameters validation")
    assert file_path or bucket or object_name, "File name is empty"
    #print(file_path, bucket, object_name)


    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_path

    # Upload the file
    this_response = None
    s3 = boto3.resource('s3')


    #print("Connect to S3 API")
    try:
        with open(file_path, 'rb') as f:
            this_response = s3.Bucket('mturk-s3-cg').put_object(Key=object_name, Body=f,ContentType='image/jpeg',ACL='public-read')
        #print("Uploading to S3 is complete!", this_response)
    except ClientError as e:
        logging.error(e)
        return False
    return True

### Get source folder to upload sub images from for classification HIT tasks

In [3]:
path = Path(os.getcwd())

#get new folder's storage entry folder, there on, folders likely similar ?
filedir = path.joinpath(path, "task1")
if os.path.exists(filedir):
    print("File directory to upload is here %s"%str(filedir))
else:
    assert os.path.exists(filedir),"Image files directory does not exist. Please check and give correct path. %s"%str(filedir)

main_folders = [filedir.joinpath(name) for name in os.listdir(filedir)] 

results = dict()
this_directory = None

#load json config
config_file_path =os.path.join("Enter correct path of config folder","config", "config.json")
if  os.path.exists(config_file_path):
    with open(config_file_path, "r") as jsonfile:
        config = json.load(jsonfile)
        print(config)
        assert config, "Config JSON has errors or is empty."
        
bucketname = config["bucketname"]
classify_images_folder = "task1/batch100/"

print("Config file loaded. %s"%str(config_file_path))

File directory to upload is here C:\Users\exx\Documents\lab\code\classification\notebooks\task1
{'overlap': 0.2, 'tile_size': 400, 'class_thumbnail_sizes': {'bus': 171, 'van_rv': 127, 'small': 101, 'specialized': 111, 'truck': 223, 'trailer_large': 219, 'trailer_small': 101, 'unknown': 127}, 'marker_color': [200, 200, 200, 255], 'point_color': [255, 0, 0], 'bb_color': [255, 0, 0], 'bucketname': 'mturk-s3-cg', 'classify_images_folder': 'task2-images/LINZ/', 'bb_images_folder': 'task1-images/LINZ/', 's3_url': 'https://mturk-s3-cg.s3.amazonaws.com/'}
Config file loaded. C:\Users\exx\Documents\lab\code\config\config.json


#### For each main image folder name, get all sub-images and upload each one of them to S3

In [7]:
count = 0
for f in main_folders:
    this_key = None
    for root, dir, files in os.walk(f):
        #ignore ipynb_checkpoints folders
        if re.search(".ipynb_checkpoints",os.path.basename(Path(root))):
            continue
        print(root, len(files))

        for file in files:
            foldername = os.path.basename(root)
            filename = os.path.join(root, file)
            object_name = classify_images_folder+foldername+"/"+os.path.basename(filename)
            if upload_file(filename, bucketname, object_name):
                count += 1
                        

C:\Users\exx\Documents\lab\code\classification\notebooks\task1\0001_0001 11
C:\Users\exx\Documents\lab\code\classification\notebooks\task1\0001_0002 26
C:\Users\exx\Documents\lab\code\classification\notebooks\task1\0001_0003 70
C:\Users\exx\Documents\lab\code\classification\notebooks\task1\0001_0004 77
C:\Users\exx\Documents\lab\code\classification\notebooks\task1\0001_0005 9
C:\Users\exx\Documents\lab\code\classification\notebooks\task1\0001_0006 19


In [8]:
count

212